In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import gzip
from IPython.display import display

data = ['../BigContest_data/' + file for file in os.listdir('../BigContest_data')]

In [6]:
# user_spec = pd.read_csv(data[0])
log_data = pd.read_csv(data[1])
loan_result = pd.read_csv(data[2])

In [7]:
user_spec = pd.read_csv('../processed/user_spec_complete_except4.csv')
with gzip.open('../processed/user_spec_filled.pickle', 'rb') as f:
    user_spec_filled = pickle.load(f)
with gzip.open('../processed/all_info.pickle', 'rb') as f:
    all_info = pickle.load(f)

In [8]:
loan_result_train = pd.read_csv('../processed/loan_result_train.csv')
loan_result_test = pd.read_csv('../processed/loan_result_test.csv')
concat_log = pd.read_csv('../processed/concat_log_complete2.csv')
pattern_binary = pd.read_csv('../processed/pattern_binary.csv')
# user_spec = pd.read_csv('../processed/user_spec_complete_except4.csv')

In [9]:
def split_comma(x):
    x_str = str(int(x))
    first_comma = len(x_str)%3
    if first_comma==0:
        comma_str = ''
    else:
        comma_str = x_str[:first_comma] + ','
    remain_str = x_str[first_comma:]
    while len(remain_str)>=3:
        comma_str += remain_str[:3]+','
        remain_str = remain_str[3:]
    return comma_str[:-1]

# log data에 대한 feature

In [ ]:
concat_log.head()

In [ ]:
concat_log['event'].unique()

In [13]:
valid_actions = ['UseLoanManage','CompleteIDCertification','UsePrepayCalc','UseDSRCalc','GetCreditInfo','is_applied_Y']
cond = concat_log['event'].isin(valid_actions)
concat_log_cond = concat_log[cond]

In [14]:
def value_counts(x,extra_indexes,order,normalize=True):
    value_counted = pd.Series(x).value_counts(normalize=normalize)
    for extra_index in extra_indexes:
        if extra_index not in value_counted.index:
            value_counted[extra_index] = 0
    value_counted = value_counted.sort_index()
    value_counted.index = value_counted.index +'_' + order
    return value_counted

In [15]:
def freq_count(df):
    is_applied_Y_index = np.where(df['event']=='is_applied_Y')[0]
    if len(is_applied_Y_index)>=2:
        first = is_applied_Y_index[0]
        last = is_applied_Y_index[-1]
        first_action = value_counts(df['event'].values[:first],valid_actions,'first')
        last_action = value_counts(df['event'].values[first:last],valid_actions,'last')
    elif len(is_applied_Y_index)==1:
        first = is_applied_Y_index[0]
        first_action = value_counts(df['event'].values[:first],valid_actions,'first')
        last_action = pd.Series([0]*len(valid_actions), index=valid_actions).sort_index()
        last_action.index = last_action.index +'_last'
    else:
        first_action = pd.Series([0]*len(valid_actions), index=valid_actions).sort_index()
        last_action = pd.Series([0]*len(valid_actions), index=valid_actions).sort_index()
        first_action.index = first_action.index +'_first'
        last_action.index = last_action.index +'_last'
    actions_freq = first_action.append(last_action)
    return actions_freq

In [18]:
# 주의 : 30분(?) 가량 걸림
import time
start = time.time()
freq_action = concat_log_cond.groupby('user_id').apply(freq_count)
print((time.time() - start)/60,'분')

30.51589287916819 분


In [ ]:
freq_action.head()

In [ ]:
import pickle
import gzip
with gzip.open('../processed/freq_action.pickle','wb') as f:
    pickle.dump(freq_action, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
with gzip.open('../processed/freq_action.pickle','rb') as f:
    freq_action = pickle.load(f)

# user_spec과의 합치는 방안 고려

In [8]:
filled_columns = ['credit_score', 'yearly_income','income_type', 'employment_type', 'houseown_type',
'company_enter_month','personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn']
user_spec[filled_columns] = user_spec_filled[filled_columns]

In [ ]:
# user_spec에는 있으나 log_data 순서 feature에는 없는 user_id
# 해당 행들은 결측치로 채워질 것임
len(np.setdiff1d(user_spec['user_id'],freq_action.index.tolist()))

In [ ]:
# log_data 순서 feature에는 있으나 user_spec에는 없는 user_id
len(np.setdiff1d(freq_action.index.tolist(),user_spec['user_id']))

In [ ]:
split_comma(user_spec.shape[0]), split_comma(all_info.shape[0]), split_comma(loan_result.shape[0])

In [311]:
all_info['user_id'] = user_spec['user_id']

In [315]:
all_info = pd.merge(all_info, freq_action, left_on='user_id', right_index=True, how='left')

# is_applied 종속변수로 추가
- 6월 날짜는 빼는 과정 거칠 것

In [ ]:
loan_result_train.query('is_applied==1').tail()

In [ ]:
# 다음과 같이 application_id가 1이 포함돼 있으면 일단 all_info에 1로 추가
# bank_id, product_id, loan_limit, loan_rate 역시 고려대상이지만 application_id 하나당 다음 여러 행을 포함하도록 병합하면 데이터사이즈가 너무 커질 것 -> 학습 시간이 어마어마할듯
# 하나의 is_applied만 포함한 데이터로 feature selection 후 bank_id, product_id, loan_limit, loan_rate 고려하는 방식이 어떨지
loan_result_train.query('application_id == 1369315')

In [11]:
# 아직 6월의 NaN은 고려하지 않음
# 0과 6월 데이터는 모두 nan으로 고려
applied_application_id = loan_result_train.query('is_applied==1')['application_id'].unique()
all_info_is_applied = all_info['application_id'].map(lambda x:1 if x in applied_application_id else np.nan)
all_info['is_applied'] = all_info_is_applied

# 기본 속성과 범주별 대표값 속성 concat

In [ ]:
user_spec.shape, all_info.shape

In [ ]:
user_spec.head()

In [21]:
# 입사년차 월 단위로 추가해주기
enter_period_year = pd.to_datetime(user_spec['insert_time']).dt.year - pd.to_datetime(user_spec['company_enter_month']).dt.year
enter_period_year *= 12
enter_period_month = pd.to_datetime(user_spec['insert_time']).dt.month - pd.to_datetime(user_spec['company_enter_month']).dt.month
enter_period = enter_period_year+enter_period_month
enter_period /= 12
enter_age = pd.to_datetime(user_spec['company_enter_month']).dt.year - user_spec['birth_year'].fillna(2022) + 1
enter_period[enter_age <= 15] = np.nan

In [25]:
# 나이변수 추가해주기
age = pd.to_datetime(user_spec['insert_time']).dt.year - user_spec['birth_year'] + 1

In [26]:
user_spec['company_period_month'] = enter_period
user_spec['age'] = age

In [35]:
all_att = pd.concat([user_spec.reset_index(drop=True), all_info.drop('application_id',axis=1).reset_index(drop=True)],axis=1)